In [ ]:
'''
Stepper motor work by
On controller level input only step and direction
Controll(step, direction)

On driver level it's send pulse signal to the magnetic coils

Opened loop could miss step
We add angular encoder to the driver and correct this making it closed loop

on controller lever it use PID control to check the position and recalculates the control signa until the target is reached.
'''